In [3]:
from tqdm import tqdm
import numpy as np
import networkx as nx
import os

from Qommunity.samplers.hierarchical.advantage_sampler import AdvantageSampler
# from Qommunity.samplers.hierarchical.gurobi_sampler import GurobiSampler
from Qommunity.searchers.hierarchical_community_searcher import HierarchicalCommunitySearcher



def make_dir(dir):
    if not os.path.exists(dir):
        os.mkdir(dir)



MIN_NODES = 10
MAX_NODES = 100

num_nodes = np.linspace(MIN_NODES, MAX_NODES, MAX_NODES//MIN_NODES)


dir = "networks/powerlaw_m=1_p=0.2"
os.makedirs(os.path.dirname(dir+"/"), exist_ok=True)

solver = "advantage"


try:
    Graphs = np.load(f"{dir}/graphs.npy", allow_pickle=True)
except:
    Graphs = np.empty(shape=(len(num_nodes),), dtype=object)
    for i, n in enumerate(num_nodes):
        Graphs[i] = nx.powerlaw_cluster_graph(n=n, m=1, p=0.2)
    np.save(f"{dir}/graphs.npy", Graphs)




dir = "profiling"
os.makedirs(os.path.dirname(dir+"/"), exist_ok=True)
make_dir(f"{dir}/{solver}")

In [4]:
N_RUNS = 1

G_idx = 1

def run_adv():
    mods = np.zeros((num_nodes[:G_idx].shape[0], N_RUNS))
    comms = np.empty((num_nodes[:G_idx].shape[0], N_RUNS), dtype=object)

    mods_graph_N = np.zeros((N_RUNS))
    comms_graph_N = np.empty((N_RUNS), dtype=object)
    for i, G in tqdm(enumerate(Graphs[:G_idx])):
        current_net_size = int(G.number_of_nodes())    
        for r in range(N_RUNS):
            # new instance of the sampler each time
            advantage = AdvantageSampler(G, num_reads=100)
            hierch_searcher = HierarchicalCommunitySearcher(advantage)
            comms_res = hierch_searcher.hierarchical_community_search()
            
            mod_score = nx.community.modularity(G, comms_res)
            mods_graph_N[r] = mod_score
            comms_graph_N[r] = comms_res

            try:
                np.save(f"{dir}/{solver}/{solver}-network_size_{current_net_size}", mods_graph_N)
                np.save(f"{dir}/{solver}/{solver}-network_size_{current_net_size}_comms", comms_graph_N)
            except Exception as e:
                print(f"iter: {i} run: {r} saving failed")
                print(e)

        # current_net_size = int(num_nodes[i])
        current_net_size = int(G.number_of_nodes())
        mods[i] = mods_graph_N
        comms[i] = comms_graph_N
        np.save(f"{dir}/{solver}/{solver}-network_size_{current_net_size}", mods_graph_N)
        np.save(f"{dir}/{solver}/{solver}-network_size_{current_net_size}_comms", comms_graph_N)

    np.save(f"{dir}/{solver}/{solver}-mods_final", mods)
    np.save(f"{dir}/{solver}/{solver}-comms_final", comms)

In [7]:
%prun -l 7 -s cumulative run_adv()

1it [00:09,  9.54s/it]

         2517294 function calls (2459329 primitive calls) in 9.541 seconds

   Ordered by: cumulative time
   List reduced from 1305 to 7 due to restriction <7>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    9.541    9.541 {built-in method builtins.exec}
        1    0.000    0.000    9.541    9.541 2671447358.py:5(run_adv)
        1    0.000    0.000    9.527    9.527 hierarchical_community_searcher.py:40(hierarchical_community_search)
      5/1    0.000    0.000    9.527    9.527 hierarchical_community_searcher.py:126(_hierarchical_search_recursion)
        5    0.015    0.003    9.517    1.903 advantage_sampler.py:34(sample_qubo_to_dict)
        5    0.000    0.000    9.502    1.900 advantage.py:123(solve)
       75    0.000    0.000    5.728    0.076 threading.py:589(wait)